# 1. Dimensionality Reduction

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap.umap_ as umap
import re
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from pathlib import Path
import plotly.graph_objects as go
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# Set random seeds for reproducibility
np.random.seed(42)

In [ ]:
save_dir = "/home/jen-hungwang/Documents/mnp-liver/results/"
# save_dir = "/storage/jenhung/results/mnp/" # Adjusted for Docker environment

## Load data

In [ ]:
# Load data
csv_dir = Path("/home/jen-hungwang/Documents/mnp-liver/csv")  # Adjusted for local environment
# csv_dir = Path("/storage/jenhung/data/mnp_liver")  # Adjusted for Docker environment
hep_path = csv_dir / "hep" # hep or huh
csv_data = "df_SingleCell_AO_HEPG2_110341.csv" # "df_HUH7_SingleCell_102912.csv", "df_HUH7_SingleCell_110341.csv", "df_HUH7_SingleCell_191735.csv"
f = hep_path / csv_data
metadata_column = "Metadata_concentration_perliter"  # Adjusted for local environment

df = pd.read_csv(f, sep=",", header=0, dtype={metadata_column: 'string'})
# df = pd.read_csv(f, sep=",", header=0)

In [ ]:
df.head()

## Randomly drop NC for data balancing

In [ ]:
value_counts = df[metadata_column].value_counts()
num_classes = len(value_counts)
print(f"Unique values and their counts in column '{metadata_column}':")
print(value_counts)
print(f"Number of classes: {num_classes}")

In [ ]:
percentage_to_keep = 50

# Identify rows where the column is exactly the string '0'
zero_mask = df[metadata_column] == '0'

num_zero_rows = zero_mask.sum()
rows_to_keep = int(num_zero_rows * percentage_to_keep / 100)

print(f"Number of rows with {metadata_column} == '0': {num_zero_rows}")
print(f"Number of rows to keep ({percentage_to_keep:.2f}% of zeros): {rows_to_keep}")

# Sample rows from '0's only if any exist
if rows_to_keep > 0:
    zero_indices = df.index[zero_mask]
    zero_indices_to_keep = np.random.default_rng(42).choice(zero_indices, size=rows_to_keep, replace=False)
    keep_mask = ~zero_mask
    keep_mask.loc[zero_indices_to_keep] = True
else:
    keep_mask = ~zero_mask

# Filter and reset index
df_filtered = df.loc[keep_mask].reset_index(drop=True)

# Check result
value_counts = df_filtered[metadata_column].value_counts()
print(f"Unique values and their counts in column '{metadata_column}' after dropping {100 - percentage_to_keep:.2f}% of '0' entries:")
print(value_counts)


In [ ]:
# Explicitly delete df to free memory
del df

## Data preprocessing

In [ ]:
def preprocess_dataframe(df, nan_threshold=0.0001):
    # Select feature columns using a generator to avoid list storage
    feature_columns = [col for col in df.columns if not (col.startswith(('Metadata_', 'Image_')) or col.endswith('_ObjectNumber'))]
    print(f"Number of feature columns: {len(feature_columns)}")
    
    # Convert to float32 upfront to reduce memory (assuming numerical data)
    X = df[feature_columns].astype('float32', copy=False)
    
    # Calculate NaN and Inf counts in one pass
    nan_counts = X.isna().sum()
    inf_counts = np.isinf(X).sum()
    
    # Identify columns with NaN or Inf exceeding threshold
    threshold = X.shape[0] * nan_threshold
    invalid_columns = set(nan_counts[nan_counts >= threshold].index).union(
        inf_counts[inf_counts >= threshold].index
    )
    valid_columns = [col for col in feature_columns if col not in invalid_columns]
    
    # Print columns with NaN or Inf if any
    columns_with_nan_or_inf = set(nan_counts[nan_counts > 0].index).union(
        inf_counts[inf_counts > 0].index
    )
    if columns_with_nan_or_inf:
        print("\nColumns with at least one NaN or Inf value:")
        print(f"{'Column':<60} {'NaN Count':>10} {'Inf Count':>10}")
        print("-" * 80)
        for col in sorted(columns_with_nan_or_inf):
            print(f"{col:<60} {nan_counts[col]:>10} {inf_counts[col]:>10}")
        print(f"Total columns with NaN or Inf: {len(columns_with_nan_or_inf)}")
    
    print(f"Number of valid columns: {len(valid_columns)}")
    if not valid_columns:
        raise ValueError("No valid columns remain after filtering.")
    
    # Select valid columns in-place
    X.drop(columns=[col for col in X.columns if col not in valid_columns], inplace=True)
    
    # Replace inf with NaN in-place
    X.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    # Fill NaN with median in one pass
    medians = X.median()
    X.fillna(medians, inplace=True)
    
    # Check for remaining NaN values
    nan_count_after_fill = X.isna().sum().sum()
    print(f"\nNaN count after filling with median: {nan_count_after_fill}")
    if nan_count_after_fill > 0:
        print("Warning: Some NaN values remain. Filling with zero.")
        X.fillna(0, inplace=True)
    
    # Check if data is valid
    if X.shape[0] == 0 or X.shape[1] == 0:
        raise ValueError("No rows/columns remain after preprocessing.")
    
    # Scale the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X).astype('float32')  # Keep float32 for scaled data
    
    # Explicitly delete X to free memory
    del X
    
    return X_scaled, valid_columns

In [ ]:
# Preprocess DataFrame
dataframes = {'df': df_filtered}
preprocessed_data = {}

for name, df in dataframes.items():
    print(f"Original {name} shape: {df.shape}")
    print(f"Preprocessing {name}...")
    X_scaled, valid_columns = preprocess_dataframe(df)
    preprocessed_data[name] = {'X_scaled': X_scaled, 'valid_columns': valid_columns, 'df': df}
    print(f"Preprocessed {name} with {len(valid_columns)} valid columns.\n")
    
    # Delete df_filtered if not needed later
    del df_filtered

## 2D & 3D Plot

In [ ]:
import re
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import umap
import seaborn as sns

def convert_concentration(value):
    if pd.isna(value) or value == "":
        return 0.0
    try:
        value = str(value).lower().replace(" ", "")
        match = re.match(r'(\d*\.?\d+)([mnu]?g)?', value)
        if match:
            num = float(match.group(1))
            unit = match.group(2) or ''
            if unit == 'mg':
                return num * 1e-3
            elif unit == 'ug':
                return num * 1e-6
            elif unit == 'ng':
                return num * 1e-9
            return num  # 'g' or no unit
        return float(value)
    except ValueError:
        return 0.0

def plot_dimensionality_reduction(X_scaled, df, valid_columns, metadata_column, method_name, title, 
                                  tsne_perplexity=30, n_neighbors=15, min_dist=0.1, continuous=True, 
                                  n_components=2, save_path=None):
    # Convert X_scaled to float32 to reduce memory
    X_scaled = X_scaled.astype('float32', copy=False)
    
    # Initialize reducer and labels
    if method_name == 'PCA':
        reducer = PCA(n_components=n_components)
        x_label, y_label = 'PC1', 'PC2'
        z_label = 'PC3' if n_components == 3 else None
    elif method_name == 't-SNE':
        reducer = TSNE(n_components=n_components, perplexity=tsne_perplexity, learning_rate='auto', random_state=42)
        x_label, y_label = 't-SNE 1', 't-SNE 2'
        z_label = 't-SNE 3' if n_components == 3 else None
    elif method_name == 'UMAP':
        reducer = umap.UMAP(n_components=n_components, n_neighbors=n_neighbors, min_dist=min_dist, random_state=42)
        x_label, y_label = 'UMAP 1', 'UMAP 2'
        z_label = 'UMAP 3' if n_components == 3 else None
    elif method_name == 'LDA':
        labels = df[metadata_column].astype('string', copy=False)
        reducer = LDA(n_components=n_components)
        x_label, y_label = 'LD1', 'LD2'
        z_label = 'LD3' if n_components == 3 else None
    else:
        raise ValueError("Unsupported method")

    # Perform dimensionality reduction
    try:
        X_reduced = (reducer.fit_transform(X_scaled, labels) if method_name == 'LDA' 
                     else reducer.fit_transform(X_scaled)).astype('float32')
    except Exception as e:
        print(f"Error in {method_name} reduction: {e}")
        return None

    # Initialize figure based on n_components
    is_3d = n_components == 3
    fig = go.Figure()
    title_suffix = ' (Continuous)' if continuous else ' (Categorical)'
    
    # Plotting
    if continuous:
        concentrations = df[metadata_column].apply(convert_concentration).astype('float32')
        if is_3d:
            fig.add_trace(go.Scatter3d(
                x=X_reduced[:, 0], y=X_reduced[:, 1], z=X_reduced[:, 2], mode='markers',
                marker=dict(color=concentrations, colorscale='Viridis', showscale=True, 
                           colorbar=dict(title=f'{metadata_column} (g)')))
            )
        else:
            fig.add_trace(go.Scatter(
                x=X_reduced[:, 0], y=X_reduced[:, 1], mode='markers',
                marker=dict(color=concentrations, colorscale='Viridis', showscale=True, 
                           colorbar=dict(title=f'{metadata_column} (g)')))
            )
    else:
        labels = df[metadata_column].astype('string', copy=False)
        unique_labels = sorted(labels.unique(), key=convert_concentration, reverse=True)
        color_map = {label: f'rgb({int(r*255)}, {int(g*255)}, {int(b*255)})' 
                     for label, (r, g, b) in zip(unique_labels, sns.color_palette('tab10', len(unique_labels)))}
        
        for label in unique_labels:
            mask = labels == label
            if is_3d:
                fig.add_trace(go.Scatter3d(
                    x=X_reduced[mask, 0], y=X_reduced[mask, 1], z=X_reduced[mask, 2], 
                    mode='markers', marker=dict(color=color_map[label], size=3),
                    name=str(label), showlegend=True
                ))
            else:
                fig.add_trace(go.Scatter(
                    x=X_reduced[mask, 0], y=X_reduced[mask, 1], 
                    mode='markers', marker=dict(color=color_map[label], size=3),
                    name=str(label), showlegend=True
                ))

        fig.update_layout(showlegend=True, legend=dict(itemsizing='constant'))

    # Update layout
    layout = dict(
        title=f"{title}{title_suffix} ({'3D' if is_3d else '2D'}, {len(valid_columns)} features)",
        **({'scene': dict(xaxis_title=x_label, yaxis_title=y_label, zaxis_title=z_label)} if is_3d 
           else {'xaxis_title': x_label, 'yaxis_title': y_label})
    )
    fig.update_layout(**layout)

    # Save plot if save_path is provided
    if save_path:
        fig.write_image(f"{save_path}_{'3D' if is_3d else '2D'}.png", width=1200, height=800)

    # Free memory
    del X_reduced
    if continuous:
        del concentrations
    else:
        del labels
    return fig

## LDA

In [ ]:
# Apply dimensionality reduction methods to each DataFrame
method = 'LDA'

for name, data in preprocessed_data.items():
    if metadata_column in data['df'].columns:
        # 2D plots
        # plot_dimensionality_reduction(data['X_scaled'], data['df'], data['valid_columns'], metadata_column,
        #                                 method, f"{method} of {csv_data}", continuous=True, n_components=2)
        plot_dimensionality_reduction(data['X_scaled'], data['df'], data['valid_columns'], metadata_column,
                                      method, f"{method} of {csv_data}", continuous=False, n_components=2, 
                                      save_path=save_dir + f"{csv_data}_{method}")
        
        # 3D plots
        # plot_dimensionality_reduction(data['X_scaled'], data['df'], data['valid_columns'], metadata_column,
        #                                 method, f"{method} of {csv_data}", continuous=True, n_components=3)
        fig_original = plot_dimensionality_reduction(data['X_scaled'], data['df'], data['valid_columns'], metadata_column,
                                      method, f"{method} of {csv_data}", continuous=False, n_components=3,
                                      save_path=save_dir + f"{csv_data}_{method}")
    else:
        print(f"Warning: {metadata_column} not found in {name}.")

# 3. Clustering

#### Grid Search LDA Dimension

In [ ]:
import os
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Function to convert concentration values to numerical for sorting
def convert_concentration(value):
    if pd.isna(value) or value == "":
        return 0.0
    try:
        value = str(value).lower().replace(" ", "")  # Normalize input
        # Use regex to extract number and unit
        match = re.match(r'(\d*\.?\d+)([mnu]?g)?', value)
        if match:
            num = float(match.group(1))
            unit = match.group(2) or ''
            if unit == 'mg':
                return num * 1e-3  # Convert mg to g
            elif unit == 'ug':
                return num * 1e-6  # Convert ug to g
            elif unit == 'ng':
                return num * 1e-9  # Convert ng to g
            elif unit == 'g' or not unit:  # Includes pure numbers or 'g'
                return num
        return float(value)  # Fallback for pure numbers
    except ValueError:
        return 0.0  # Default to 0 for invalid entries
    
# Configuration
range_n_clusters = [3]
range_n_components = [4] # [2, 3, 4, 5, 6, 7]
use_gmm = True
gmm_covariance_types = ['full', 'tied'] # ['full', 'tied', 'diag', 'spherical']

for name, data in preprocessed_data.items():
    if metadata_column not in data['df'].columns:
        print(f"Warning: {metadata_column} not found in {name}.")
        continue

    # Use the DataFrame directly, avoiding copy
    df = data['df']
  
    # Standardize '0' values in-place to string '0' for consistency
    df[metadata_column] = df[metadata_column].apply(lambda x: x.lower().strip())

    X_scaled = data['X_scaled'].astype('float32', copy=False)  # Ensure float32

    for n_comp in range_n_components:
        try:
            reducer = LDA(n_components=n_comp)
            X_reduced = reducer.fit_transform(X_scaled, df[metadata_column])
        except ValueError as e:
            print(f"Error with n_components={n_comp} for {name}: {e}")
            continue

        # Iterate over covariance types if using GMM
        covariance_iter = gmm_covariance_types if use_gmm else [None]
        for gmm_covariance in covariance_iter:
            for n_clusters in range_n_clusters:
                # Create figure with three subplots
                fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 7), gridspec_kw={'width_ratios': [1, 1, 1.5]})

                # Clustering
                try:
                    if use_gmm:
                        clusterer = GaussianMixture(
                            n_components=n_clusters, random_state=10, covariance_type=gmm_covariance
                        )
                        algo_name = f"GMM_{gmm_covariance}"
                    else:
                        clusterer = KMeans(n_clusters=n_clusters, random_state=10)
                        algo_name = "KMeans"

                    cluster_labels = clusterer.fit_predict(X_reduced)
                    centers = clusterer.means_ if use_gmm else clusterer.cluster_centers_
                    centers = centers.astype('float32')  # Reduce memory
                except Exception as e:
                    print(f"Error in clustering for {name}, n_components={n_comp}, n_clusters={n_clusters}, {algo_name}: {e}")
                    plt.close(fig)
                    continue

                silhouette_avg = silhouette_score(X_reduced, cluster_labels)
                print(
                    f"For {name}, n_components={n_comp}, n_clusters={n_clusters}, {algo_name}, "
                    f"silhouette_score={silhouette_avg:.4f}"
                )

                # Silhouette Plot (ax1)
                ax1.set_xlim([-0.1, 1])
                ax1.set_ylim([0, len(X_reduced) + (n_clusters + 1) * 10])
                sample_silhouette_values = silhouette_samples(X_reduced, cluster_labels)

                y_lower = 10
                for i in range(n_clusters):
                    ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
                    ith_cluster_silhouette_values.sort()
                    size_cluster_i = ith_cluster_silhouette_values.shape[0]
                    y_upper = y_lower + size_cluster_i
                    color = cm.nipy_spectral(float(i) / n_clusters)
                    ax1.fill_betweenx(
                        np.arange(y_lower, y_upper),
                        0,
                        ith_cluster_silhouette_values,
                        facecolor=color,
                        edgecolor=color,
                        alpha=0.7,
                    )
                    ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
                    y_lower = y_upper + 10

                ax1.set_title("Silhouette Plot")
                ax1.set_xlabel("Silhouette Coefficient Values")
                ax1.set_ylabel("Cluster Label")
                ax1.axvline(x=silhouette_avg, color="red", linestyle="--")
                ax1.set_yticks([])
                ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

                # Cluster Scatter Plot (ax2)
                colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
                ax2.scatter(
                    X_reduced[:, 0], X_reduced[:, 1], marker=".", s=30, lw=0,
                    alpha=0.7, c=colors, edgecolor="k"
                )
                ax2.scatter(
                    centers[:, 0], centers[:, 1], marker="o", c="white", alpha=1,
                    s=200, edgecolor="k"
                )
                for i, c in enumerate(centers):
                    ax2.scatter(c[0], c[1], marker=f"${i}$", alpha=1, s=50, edgecolor="k")

                ax2.set_title(f"Cluster Visualization ({algo_name})")
                ax2.set_xlabel("Feature Space (1st Dimension)")
                ax2.set_ylabel("Feature Space (2nd Dimension)")

                # Concentration Distribution Plot (ax3)
                unique_clusters = np.unique(cluster_labels[cluster_labels != -1])
                concentration_values = df[metadata_column].unique()
                converted_values = np.array([convert_concentration(val) for val in concentration_values])
                sort_indices = np.argsort(converted_values)[::-1]
                sorted_concentration_values = concentration_values[sort_indices]

                for idx, cluster in enumerate(unique_clusters):
                    cluster_mask = cluster_labels == cluster
                    cluster_concentrations = df[metadata_column][cluster_mask]
                    counts = np.zeros(len(sorted_concentration_values), dtype='int32')  # Use int32
                    for i, conc in enumerate(sorted_concentration_values):
                        counts[i] = np.sum(cluster_concentrations == conc)

                    ax3.bar(
                        np.arange(len(sorted_concentration_values)) + idx * 0.2,
                        counts,
                        width=0.2,
                        label=f'Cluster {cluster}',
                        color=cm.nipy_spectral(float(cluster) / n_clusters),
                        alpha=0.7
                    )

                ax3.set_title("Concentration Distribution Across Clusters")
                ax3.set_xlabel("Concentration Levels (High to Low)")
                ax3.set_ylabel("Count")
                ax3.set_xticks(np.arange(len(sorted_concentration_values)))
                ax3.set_xticklabels(sorted_concentration_values, rotation=45, ha='right')
                ax3.legend()

                # Overall figure title
                plt.suptitle(
                    f"{csv_data} - Analysis with {algo_name}, Silhouette={silhouette_avg:.4f}, "
                    f"n_clusters={n_clusters}, LDA={n_comp}D",
                    fontsize=14, fontweight="bold",
                )

                # Save and close plot
                filename = f"{algo_name}_{n_clusters}clusters_{n_comp}D_{csv_data}.png"
                plt.savefig(os.path.join(save_dir, filename), bbox_inches='tight', dpi=300)
                plt.close(fig)  # Free memory
                del fig  # Explicitly delete figure

        # Free memory after each n_components iteration
        del X_reduced

## (Optional) Grid Search: GMM

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm

def gmm_bic_score(estimator, X):
    return -estimator.bic(X)

# Configuration
param_grid = {
    "lda__n_components": range(2, 4),
    "gmm__n_components": range(2, 4),
    "gmm__covariance_type": ['full', 'tied'] # , 'diag', 'spherical']
}
metadata_column = 'Metadata_concentration_perliter'
eval_method = 'Silhouette'  # 'BIC', 'Silhouette'
results = []

# Grid search
for name, data in preprocessed_data.items():
    if metadata_column not in data['df'].columns:
        print(f"Warning: {metadata_column} not found in {name}.")
        continue

    X_scaled = data['X_scaled'].astype('float32', copy=False)
    labels = data['df'][metadata_column].astype('string', copy=False)

    for params in tqdm(ParameterGrid(param_grid), desc=f"Grid Search in {name}"):
        try:
            # Apply LDA
            n_lda = params['lda__n_components']
            reducer = LDA(n_components=n_lda)
            X_reduced = reducer.fit_transform(X_scaled, labels).astype('float32')

            # Fit GMM
            gmm_params = {
                'n_components': params['gmm__n_components'],
                'covariance_type': params['gmm__covariance_type'],
                'random_state': 42
            }
            gmm = GaussianMixture(**gmm_params)
            
            if eval_method == 'Silhouette':
                cluster_labels = gmm.fit_predict(X_reduced)
                score = (silhouette_score(X_reduced, cluster_labels) 
                         if len(set(cluster_labels)) > 1 and -1 not in set(cluster_labels) 
                         else float('-inf'))
            else:  # BIC
                gmm.fit(X_reduced)
                score = gmm_bic_score(gmm, X_reduced)

            print(f"Parameters: {params}, {eval_method} Score: {score:.4f}")
            results.append({
                'lda__n_components': n_lda,
                'gmm__n_components': params['gmm__n_components'],
                'gmm__covariance_type': params['gmm__covariance_type'],
                'score': score,
                'dataset': name
            })

            # Free memory
            del X_reduced, gmm
        except Exception as e:
            print(f"Error in {name} with params {params}: {e}")

# Create DataFrame from results
df_lda = pd.DataFrame(results)
if eval_method == 'BIC':
    df_lda['score'] = -df_lda['score']  # Convert negative BIC to positive

# Rename columns
df_lda = df_lda.rename(columns={
    'lda__n_components': 'Number of components (LDA)',
    'gmm__n_components': 'Number of clusters (GMM)',
    'gmm__covariance_type': 'Covariance Type',
    'score': f'{eval_method} score'
})

# Find best parameters
if results:
    best_result = df_lda.loc[df_lda[f'{eval_method} score'].idxmax()]
    print("Best overall:")
    print(f"Parameters: LDA components={best_result['Number of components (LDA)']}, "
          f"GMM clusters={best_result['Number of clusters (GMM)']}, "
          f"Covariance={best_result['Covariance Type']}, Dataset={best_result['dataset']}")
    print(f"Best {eval_method} score: {best_result[f'{eval_method} score']:.4f}")

# Create and save visualization
if not df_lda.empty:
    g = sns.catplot(
        data=df_lda,
        kind="bar",
        x="Number of components (LDA)",
        y=f"{eval_method} score",
        hue="Number of clusters (GMM)",
        col="Covariance Type",
        col_wrap=2,
        height=4,
        aspect=1.5
    )
    g.set_axis_labels("LDA Components", f"{eval_method} Score")
    g.fig.suptitle(f"LDA vs. GMM on {csv_data}", y=1.05)
    g.set_titles("Covariance: {col_name}")
    for ax in g.axes.flat:
        ax.grid(True, axis='y', linestyle="--", alpha=0.7)

    # Save and close figure
    plt.savefig(f"{save_dir}/{csv_data}_lda_dimension_{eval_method}_scores.png", 
                dpi=300, bbox_inches="tight")
    plt.close(g.fig)
    del g

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm

def gmm_bic_score(estimator, X):
    return -estimator.bic(X)

# Configuration
param_grid = {
    "lda__n_components": range(2, 4),
    "gmm__n_components": range(2, 4),
    "gmm__covariance_type": ['full', 'tied','diag', 'spherical']
}
metadata_column = 'Metadata_concentration_perliter'
eval_method = 'Silhouette'  # 'BIC', 'Silhouette'
results = []

# Grid search
for name, data in preprocessed_data.items():
    if metadata_column not in data['df'].columns:
        print(f"Warning: {metadata_column} not found in {name}.")
        continue

    X_scaled = data['X_scaled'].astype('float32', copy=False)
    labels = data['df'][metadata_column].astype('string', copy=False)

    for params in tqdm(ParameterGrid(param_grid), desc=f"Grid Search in {name}"):
        try:
            # Apply LDA
            n_lda = params['lda__n_components']
            reducer = LDA(n_components=n_lda)
            X_reduced = reducer.fit_transform(X_scaled, labels).astype('float32')

            # Fit GMM
            gmm_params = {
                'n_components': params['gmm__n_components'],
                'covariance_type': params['gmm__covariance_type'],
                'random_state': 42
            }
            gmm = GaussianMixture(**gmm_params)
            
            if eval_method == 'Silhouette':
                cluster_labels = gmm.fit_predict(X_reduced)
                score = (silhouette_score(X_reduced, cluster_labels) 
                         if len(set(cluster_labels)) > 1 and -1 not in set(cluster_labels) 
                         else float('-inf'))
            else:  # BIC
                gmm.fit(X_reduced)
                score = gmm_bic_score(gmm, X_reduced)

            print(f"Parameters: {params}, {eval_method} Score: {score:.4f}")
            results.append({
                'lda__n_components': n_lda,
                'gmm__n_components': params['gmm__n_components'],
                'gmm__covariance_type': params['gmm__covariance_type'],
                'score': score,
                'dataset': name
            })

            # Free memory
            del X_reduced, gmm
        except Exception as e:
            print(f"Error in {name} with params {params}: {e}")

# Create DataFrame from results
df_lda = pd.DataFrame(results)
if eval_method == 'BIC':
    df_lda['score'] = -df_lda['score']  # Convert negative BIC to positive

# Rename columns
df_lda = df_lda.rename(columns={
    'lda__n_components': 'Number of components (LDA)',
    'gmm__n_components': 'Number of clusters (GMM)',
    'gmm__covariance_type': 'Covariance Type',
    'score': f'{eval_method} score'
})

# Find best parameters
if results:
    best_result = df_lda.loc[df_lda[f'{eval_method} score'].idxmax()]
    print("Best overall:")
    print(f"Parameters: LDA components={best_result['Number of components (LDA)']}, "
          f"GMM clusters={best_result['Number of clusters (GMM)']}, "
          f"Covariance={best_result['Covariance Type']}, Dataset={best_result['dataset']}")
    print(f"Best {eval_method} score: {best_result[f'{eval_method} score']:.4f}")

# Create and save visualization
if not df_lda.empty:
    g = sns.catplot(
        data=df_lda,
        kind="bar",
        x="Number of components (LDA)",
        y=f"{eval_method} score",
        hue="Number of clusters (GMM)",
        col="Covariance Type",
        col_wrap=2,
        height=4,
        aspect=1.5
    )
    g.set_axis_labels("LDA Components", f"{eval_method} Score")
    g.fig.suptitle(f"LDA vs. GMM on {csv_data}", y=1.05)
    g.set_titles("Covariance: {col_name}")
    
    # Add grid and customize bars
    for ax in g.axes.flat:
        ax.grid(True, axis='y', linestyle="--", alpha=0.7)
        # Add decimal labels to bars with .3f format
        for container in ax.containers:
            ax.bar_label(container, fmt='%.3f', label_type='edge', padding=3)
    
    # Remove default legend and add custom legend at bottom center with title
    g._legend.remove()
    g.add_legend(loc='lower center', bbox_to_anchor=(0.5, -0.05), ncol=len(df_lda['Number of clusters (GMM)'].unique()), title="Number of clusters (GMM)")
    
    # Save and close figure
    plt.savefig(f"{save_dir}/{csv_data}_lda_dimension_{eval_method}_scores.png", 
                dpi=300, bbox_inches="tight")
    plt.close(g.fig)
    del g